In [ ]:
import os, sys
sys.path.insert(1,'./scripts')
from scripts.loader import EventLoader
from scripts.chunker import Chunker
from scripts.eparser import EventParser
from scripts.exporter import Exporter
from scripts.network import Network

In [ ]:
CARS = "cars_only"
ALL = "all_transport"
ALL_WALK = "all_with_walk"

In [ ]:
xml_path = "../matsim/tramola_capacity1_100it/output_events.xml.gz"
network_path = "../matsim/tramola_capacity1_100it/output_network.xml.gz"
GTFS_path = "../PID_GTFS_1_11_2021_filtered"
vehicle_types = []

In [ ]:
def load_population(event_size=1.5*10e6):
    print("Loading population:")
    loader = EventLoader(xml_path=xml_path)
    loader.load_population_CSV(chunk_size=event_size)

    print("Parsing agents:")
    parser = EventParser(csv_files=loader.gather_csv_files())
    parser()
    print("Agents parsed.")

    chunker = Chunker()
    chunker(chunk_size=500)

def parse_population():
    loader = EventLoader(xml_path=xml_path)
    print("Parsing agents:")
    parser = EventParser(csv_files=loader.gather_csv_files())
    parser()
    print("Agents parsed.")


def load_new_chunks(chunk: int):
    chunker = Chunker()
    chunker(chunk_size=chunk)
    

def export_agents(export_mode):
    exporter = Exporter("agent", network_path=network_path, export_mode=export_mode, gtfs_path=GTFS_path)
    exporter.export_agents(format = 'json', parallel=True, proc=os.cpu_count())
    del exporter

def export_vehicles(vehicle_types):
    for veh_type in vehicle_types:
        exporter = Exporter(veh_type, network_path=network_path, export_mode=ALL, gtfs_path=GTFS_path)
        exporter.export_agents(format = 'json', parallel=True, proc=os.cpu_count())

    del exporter

In [ ]:
load_population(event_size=3*10e6) 
#~3hod for agents and cars 100k, synthpop ~1.20hod
#parse_population()

In [ ]:
load_new_chunks(200) #cars and agents 100k : 1m20s, all : 2min

In [ ]:
#export_agents(ALL) #100k = 208 min (cars), synthpop ~ 50 min (cars), synthpop ~ xx min (all)

In [ ]:
export_vehicles(["bike"]) #3 min

In [ ]:
export_vehicles(["subway"]) #3 min

In [ ]:
export_vehicles(["funicular"])

In [ ]:
export_vehicles(["car"]) #100k pop = 55 min, synthpop 15min

In [ ]:
export_vehicles(["bus"]) #20min

In [ ]:
export_vehicles(["tram"]) #7 min

In [ ]:
n = Network()
n.set_path(network_path)
n.load_nodes()
n.load_links()
n.join_network()
n.status()
n.export_shp("./output/network.shp")